In [7]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

sub_folder = 'data/Aplicativo TRB/'

In [8]:
columns_servidores = ['tiempo','servidor','path', 'metrica', 'value', 'muestreo']

In [16]:
def read_servidores(csv_name):
    data = pd.read_csv(csv_name, sep = ',', header = None)
    data.columns = columns_servidores
    
    data['tiempo'] = pd.to_datetime(data['tiempo'])
    data = data.sort_values(by=['tiempo'])

    data.set_index('tiempo', inplace=True)
    
    return data


def read_incidencias(excel_name):
    data = pd.read_excel(excel_name)
    
    #data['tiempo'] = data['Fecha'].astype(str) + ' ' + data['HORA INICIO'].astype(str)
    
    #data['tiempo'] = pd.to_datetime(
    #    data['tiempo'],errors='coerce',format='%Y/%m/%d %H:%M:%S'
    #)
    
    #data = data.sort_values(by=['tiempo'])
    #data.set_index('tiempo', inplace=True)
    
    return data

In [17]:
servidores = read_servidores(sub_folder+'TRB - Servidores - Colas Demonios - Agosto - Enero.csv')

In [18]:
incidencias = read_incidencias(sub_folder+'TRB - Lista de Alertas e Incidencias.xlsx')

In [19]:
servidores.head(3)

,servidor,path,metrica,value,muestreo
tiempo,,,,,
2019-08-02 00:00:00,prd01bl,prd01bl,QOS_COMPUTER_UPTIME,93770418.0,60.0
2019-08-02 00:00:00,caapmvmwlnxv13,caapmvmwlnxv13.lnx.gp.inet,QOS_COMPUTER_UPTIME,73237487.0,60.0
2019-08-02 00:00:31,prd01bl,AR1DDFEEDBACK,QOS_PROCESS_STATE,1.0,1.0


In [20]:
incidencias.head(3)

,ID de la incidencia*+,Apellidos+,Nombre+,Ubicación del cliente,Resumen*,Servicio*+,Prioridad*,Estado*,Grupo asignado*+,Usuario asignado+,...,Tipo de ticket,Ubicación de asistencia del contacto directo,Ubicación del contacto,Urgencia*,Usuario desconocido,VIP,Zona,Zona del contacto directo,Zona horaria del contacto directo,Última modificación por
0,INC000003495219,RAZURI CASTRO,LUZ ANDREA,Site Name por defecto,caapmvmwlnxv13 - ALERTADO,NaN,Baja,Cerrado,Operaciones,EDISON MURILLO VILLANUEVA,...,Incidencia,NaN,NaN,4-Baja,NaN,No,NaN,NaN,NaN,AR_ESCALATOR
1,INC000003494697,RAZURI CASTRO,LUZ ANDREA,Site Name por defecto,caapmvmwlnxv13 - ALERTADO,NaN,Baja,Cerrado,Operaciones,EDISON MURILLO VILLANUEVA,...,Incidencia,NaN,NaN,4-Baja,NaN,No,NaN,NaN,NaN,AR_ESCALATOR
2,INC000003494624,RAZURI CASTRO,LUZ ANDREA,Site Name por defecto,caapmvmwlnxv13 - ALERTADO,NaN,Baja,Cerrado,Operaciones,EDISON MURILLO VILLANUEVA,...,Incidencia,NaN,NaN,4-Baja,NaN,No,NaN,NaN,NaN,AR_ESCALATOR


In [23]:
servidores.groupby(['servidor','metrica','path'])['value'].std()

servidor        metrica                path                      
caapmvmwlnxv13  QOS_COMPUTER_UPTIME    caapmvmwlnxv13.lnx.gp.inet    1.725819e+07
                QOS_CPU_USAGE          caapmvmwlnxv13.lnx.gp.inet    1.286268e+00
                QOS_DISK_USAGE_PERC    /                             2.032934e+00
                                       /apm                          7.303027e-01
                                       /boot                         0.000000e+00
                                       /dev                          0.000000e+00
                                       /dev/shm                      0.000000e+00
                                       /home                         6.499859e+00
                                       /smartstor                    2.215283e+00
                                       /tmp                          1.245053e+00
                                       /trace                        5.844544e+00
                                

In [24]:
servidores_group = servidores.groupby(['servidor','metrica','path'])

In [38]:
%matplotlib qt
_,ax = plt.subplots()

for r in servidores_group.groups.keys():
    _servidor, _metrica, _path = r

    #if (_metrica != 'QOS_COMPUTER_UPTIME' and _metrica != 'QOS_MEMORY_PAGING') and _servidor == 'sun6c52':
    #if _metrica != 'QOS_COMPUTER_UPTIME' and _metrica != 'QOS_MEMORY_PAGING':
    #if _servidor == 'prd01bl' and _metrica == 'QOS_DISK_USAGE_PERC' and _path == '/':
    if _metrica == 'QOS_CPU_USAGE': 
        _df = servidores_group.get_group(r).copy()
        #_df['tiempo'] = pd.to_datetime(_df['tiempo'])
        #_df.set_index('tiempo', inplace=True)
        _df['value'].plot(ax = ax, label= _metrica+' '+_servidor+' '+_path)
        
# incidencias



ax.legend(loc='upper right')
plt.suptitle('CMS', fontsize=30)

Text(0.5, 0.98, 'CMS')

In [30]:
from adtk.data import validate_series
from adtk.visualization import plot
from adtk.detector import SeasonalAD
from adtk.detector import AutoregressionAD
from adtk.detector import GeneralizedESDTestAD

In [34]:
r = ('prd01bl', 'QOS_CPU_USAGE', 'prd01bl')
s_train = servidores_group.get_group(r).copy()['value']
s_train.head(3)

tiempo
2019-08-02 00:03:22    73.58
2019-08-02 00:08:22    65.32
2019-08-02 00:13:22    60.94
Name: value, dtype: float64

In [36]:
esd_ad = GeneralizedESDTestAD(alpha=0.3)
anomalies = esd_ad.fit_detect(s_train)
plot(s_train, anomaly_pred=anomalies, ts_linewidth=2, ts_markersize=3, ap_markersize=5, ap_color='red', ap_marker_on_curve=True);

In [37]:
seasonal_ad = SeasonalAD(freq=288)
anomalies = seasonal_ad.fit_detect(s_train)
anomalies
#plot(s_train, anomaly_pred=anomalies, ap_color='red', ap_marker_on_curve=True)

RuntimeError: Series does not follow any known frequency (e.g. second, minute, hour, day, week, month, year, etc.